In [30]:
!pip install emoji


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [31]:
import re
import numpy as np
import pandas as pd

In [32]:
df = pd.read_csv('data_pt_brazil_normalized_emoji_merged.csv',
                                converters={
                     "normalized_text_stemm_emoji": lambda x: x.strip("[]").replace("'","").split(", ")
                 })
df = df[['created_at', 'text', 'sentiment', 'normalized_text_stemm_emoji']]
df.head()

,created_at,text,sentiment,normalized_text_stemm_emoji
0,2021-07-04 22:17:21,RT @emirsader: Após ficar em silêncio na CPI d...,Positivo,"[após, fic, silênci, cpi, carl, wizard, retorn..."
1,2019-11-30 14:35:33,RT @VittorGuidoni: para de lamber fone de ouvi...,Negativo,"[lamb, fon, ouv, list, doenç, transmiss, oral,..."
2,2022-01-11 12:12:28,@exposed_exposer @CarlaZambelli38 @andrizek De...,Negativo,"[defin, vacin, tip, subst, bactér, introduz, c..."
3,2019-11-19 14:52:06,RT @canaltech: Por que o iPhone é mais seguro ...,Positivo,"[iphon, segur, contr, ]"
4,2021-12-28 23:09:10,RT @mariareinhardtt: covid tá a estragar compl...,Negativo,"[estrag, complet, tud, ]"


In [33]:
emoji_bank = {
  ",:(": "😓",
  ",:)": "😅",
  ",:-(": "😓",
  ",:-)": "😅",
  "0:)": "😇",
  "0:-)": "😇",
  "8-)": "😎",
  ":$": "😒",
  ":'(": "😢",
  ":')": "😂",
  ":'-(": "😢",
  ":'-)": "😂",
  ":'-D": "😂",
  ":'D": "😂",
  ":(": "😦",
  ":)": "😃",
  ":*": "😗",
  ":,'(": "😭",
  ":,'-(": "😭",
  ":,(": "😢",
  ":,)": "😂",
  ":,-(": "😢",
  ":,-)": "😂",
  ":,-D": "😂",
  ":,D": "😂",
  ":-$": "😒",
  ":-(": "😦",
  ":-)": "😃",
  ":-*": "😗",
  ":-/": "😕",
  ":-@": "😡",
  ":-D": "😄",
  ":-o": "😮",
  ":-O": "😮",
  ":-P": "😛",
  ":-S": "😒",
  ":-Z": "😒",
  ":-|": "😐",
  ":/": "😕",
  ":@": "😡",
  ":D": "😄",
  ":o": "😮",
  ":O": "😮",
  ":P": "😛",
  ":s": "😒",
  ":z": "😒",
  ":|": "😐",
  ";(": "😭",
  ";)": "😉",
  ";-(": "😭",
  ";-)": "😉",
  "]:)": "😈",
  "]:-)": "😈",
  "B-)": "😎",
  "o:)": "😇",
  "O:)": "😇",
  "O:-)": "😇",
  "o:-)": "😇",
  "X-)": "😆",
  "x-)": "😆"
}

In [34]:
stopwords_list = ['rt']

In [35]:
import re, string, unicodedata
import nltk
import emoji

def extract_emojis(sample):
    return ' '.join(c for c in sample if c in emoji.EMOJI_DATA).split()

def remove_user_from_text(words):
    return " ".join(filter(lambda x:x[0]!='@', words.split()))

def remove_double_space(sample):
    return " ".join(sample.split())

def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", "", sample)

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    return list(map(lambda x: x.lower(), words))

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def clean_text(sample):
    emoji_pat = '[\U0001F300-\U0001F64F\U0001F680-\U0001F6FF\u2600-\u26FF\u2700-\u27BF]'
    shrink_whitespace_reg = re.compile(r'\s{2,}')
    reg = re.compile(r'({})|[^a-zA-Z]'.format(emoji_pat)) # line a
    result = reg.sub(lambda x: ' {} '.format(x.group(1)) if x.group(1) else ' ', sample)
    return shrink_whitespace_reg.sub(' ', result)

def replace_asci_emoji(sample):
    address = sample
    for k,v in emoji_bank.items():
        address = address.replace(k, v)
    return address

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords_list:
            new_words.append(word)
    return new_words

def normalize(words):
    words = remove_user_from_text(words)
    words = to_lowercase(words.split())
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return words

def preprocess(sample):
    sample = remove_URL(sample)
    sample = replace_asci_emoji(sample)
#     sample = clean_text(sample)
    sample = remove_double_space(sample)
    return normalize(sample)

In [36]:
df['normalized'] = df.apply(lambda x: " ".join(preprocess(x['text'])), axis=1)
df.head()

,created_at,text,sentiment,normalized_text_stemm_emoji,normalized
0,2021-07-04 22:17:21,RT @emirsader: Após ficar em silêncio na CPI d...,Positivo,"[após, fic, silênci, cpi, carl, wizard, retorn...",após ficar em silêncio na cpi da covid carlos ...
1,2019-11-30 14:35:33,RT @VittorGuidoni: para de lamber fone de ouvi...,Negativo,"[lamb, fon, ouv, list, doenç, transmiss, oral,...",para de lamber fone de ouvido lista de doenças...
2,2022-01-11 12:12:28,@exposed_exposer @CarlaZambelli38 @andrizek De...,Negativo,"[defin, vacin, tip, subst, bactér, introduz, c...",definição de vacina é um tipo de substância ví...
3,2019-11-19 14:52:06,RT @canaltech: Por que o iPhone é mais seguro ...,Positivo,"[iphon, segur, contr, ]",por que o iphone é mais seguro contra vírus
4,2021-12-28 23:09:10,RT @mariareinhardtt: covid tá a estragar compl...,Negativo,"[estrag, complet, tud, ]",covid tá a estragar completamente tudo


In [37]:
df = df[['created_at', 'sentiment', 'normalized', 'normalized_text_stemm_emoji']]
df.head()

,created_at,sentiment,normalized,normalized_text_stemm_emoji
0,2021-07-04 22:17:21,Positivo,após ficar em silêncio na cpi da covid carlos ...,"[após, fic, silênci, cpi, carl, wizard, retorn..."
1,2019-11-30 14:35:33,Negativo,para de lamber fone de ouvido lista de doenças...,"[lamb, fon, ouv, list, doenç, transmiss, oral,..."
2,2022-01-11 12:12:28,Negativo,definição de vacina é um tipo de substância ví...,"[defin, vacin, tip, subst, bactér, introduz, c..."
3,2019-11-19 14:52:06,Positivo,por que o iphone é mais seguro contra vírus,"[iphon, segur, contr, ]"
4,2021-12-28 23:09:10,Negativo,covid tá a estragar completamente tudo,"[estrag, complet, tud, ]"


In [38]:
df = df.sample(n=20000, random_state=42).reset_index(drop=True)

In [39]:
df['label'] = df['sentiment'].replace(['Positivo','Negativo','Neutro'],[2,0,1])

In [40]:
from keras.utils.np_utils import to_categorical
labels = to_categorical(df['label'], num_classes=3)

In [41]:
labels

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], dtype=float32)

In [42]:
X = df['normalized_text_stemm_emoji'].apply(lambda x: ' '.join(x))
# X = df['normalized']

In [43]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X , labels, test_size=0.20, random_state=42)

In [44]:
print((X_train.shape, y_train.shape, X_test.shape, y_test.shape))

((16000,), (16000, 3), (4000,), (4000, 3))


In [45]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

In [46]:
epochs = 50
emb_dim = 256
batch_size = 256
n_most_common_words = 2000
max_len = 130

In [47]:
tokenizer = Tokenizer(num_words=n_most_common_words, lower=True)
tokenizer.fit_on_texts(df['normalized_text_stemm_emoji'].values)
sequences = tokenizer.texts_to_sequences(df['normalized_text_stemm_emoji'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = sequence.pad_sequences(sequences, maxlen=max_len)

Found 15878 unique tokens.


In [48]:
X_train, X_test, y_train, y_test = train_test_split(X , labels, test_size=0.20, random_state=42)

In [49]:
model = Sequential()
model.add(Embedding(n_most_common_words, emb_dim, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.1))
model.add(LSTM(32, dropout=0.1, recurrent_dropout=0.1))
# model.add(Dense(32, activation="relu"))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 130, 256)          512000    
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 130, 256)         0         
 lDropout1D)                                                     
                                                                 
 lstm_1 (LSTM)               (None, 32)                36992     
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                                 
Total params: 549,091
Trainable params: 549,091
Non-trainable params: 0
_________________________________________________________________
None


In [50]:
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

Epoch 1/50
50/50 [==============================] - 8s 137ms/step - loss: 0.9944 - acc: 0.5177 - val_loss: 0.9194 - val_acc: 0.5547
Epoch 2/50
50/50 [==============================] - 7s 134ms/step - loss: 0.7795 - acc: 0.6576 - val_loss: 0.7700 - val_acc: 0.6622
Epoch 3/50
50/50 [==============================] - 7s 135ms/step - loss: 0.6182 - acc: 0.7484 - val_loss: 0.7523 - val_acc: 0.6850
Epoch 4/50
50/50 [==============================] - 7s 135ms/step - loss: 0.5589 - acc: 0.7791 - val_loss: 0.7505 - val_acc: 0.6922
Epoch 5/50
50/50 [==============================] - 7s 134ms/step - loss: 0.5283 - acc: 0.7923 - val_loss: 0.7805 - val_acc: 0.6856
Epoch 6/50
50/50 [==============================] - 7s 134ms/step - loss: 0.5078 - acc: 0.8034 - val_loss: 0.7966 - val_acc: 0.6878
Epoch 7/50
50/50 [==============================] - 7s 136ms/step - loss: 0.4881 - acc: 0.8151 - val_loss: 0.8096 - val_acc: 0.6919
Epoch 8/50
50/50 [==============================] - 7s 135ms/step - loss: 0.

In [51]:
from sklearn.metrics import classification_report

preds = np.argmax(model.predict(X_test), axis = 1)
print(classification_report(np.argmax(y_test, axis = 1), preds))

125/125 [==============================] - 1s 6ms/step
              precision    recall  f1-score   support

           0       0.71      0.73      0.72      1930
           1       0.63      0.61      0.62      1191
           2       0.60      0.59      0.59       879

    accuracy                           0.66      4000
   macro avg       0.65      0.64      0.64      4000
weighted avg       0.66      0.66      0.66      4000

